<a href="https://colab.research.google.com/github/pld000/z3/blob/main/task-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


По тропинке вдоль кустов
Шли 11 хвостов.
Сосчитать я также смог,
Что шагало 30 ног.
Это вместе шли куда-то
Петухи и поросята.
А теперь вопрос таков:
Сколько было петухов?
И узнать я был бы рад,
Сколько было поросят?

In [2]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 66.5 MB/s eta 0:00:00


In [3]:
from z3 import *

In [4]:
chickenCount, pigCount = Ints('chickenCount pigCount')
equations = [
    chickenCount * 2 + pigCount * 4 == 30,
    chickenCount + pigCount == 11,
    chickenCount >= 1,
    pigCount >= 1
]
solve(equations)

[pigCount = 4, chickenCount = 7]


Три купца разделили между собой 100 монет. Первый взял в 2 раза больше, чем второй, а третий на 10 меньше, чем первый. Сколько монет взял каждый?

In [5]:
a, b, c = Ints('a b c')
solve(
    a + b + c == 100,
    a == 2 * b,
    c == a - 10,
    a > 0, b > 0, c > 0
)

[b = 22, c = 34, a = 44]


На рычаг длиной 2 метра на одно плечо на расстоянии 0.5 м от опоры положили груз массой 4 кг. На каком расстоянии от опоры на другом плече нужно положить груз массой 2 кг, чтобы рычаг был в равновесии?

In [6]:
from z3 import *

m1, m2, l1, l2 = Reals('m1 m2 l1 l2')

solve(
    m1 * l1 == m2 * l2,  # правило рычага
    m1 == 4,
    m2 == 2,
    l1 == 0.5,
    l2 > 0
)


[m2 = 2, l2 = 1, l1 = 1/2, m1 = 4]


Смешивают 2 литра воды при температуре 80 °C и некоторое количество воды при температуре 20 °C. В результате получается смесь температурой 60 °C. Сколько литров холодной воды добавили?

In [7]:
V1, V2, T1, T2, T = Reals('V1 V2 T1 T2 T')

solve(
    V1 * T1 + V2 * T2 == (V1 + V2) * T,
    V1 == 2,
    T1 == 80,
    T2 == 20,
    T == 60,
    V2 > 0
)

[V1 = 2, T1 = 80, T = 60, V2 = 1, T2 = 20]


# **Chat GPT solutions**

Классическая головоломка. У вас имеется 100 долларов, и вы можете купить 100 животных. Собаки стоят по 15 долларов, коты по 1 доллару, мыши по 25 центов. Вы должны купить хотя бы по одному животному каждого вида. Найдите решение этого паззла.

In [8]:
from z3 import *

x, y, z = Ints('x y z')

solver = Solver()
solver.add(
    x + y + z == 100,
    15 * x + y + 0.25 * z == 100,
    x >= 1,
    y >= 1,
    z >= 1
)

if solver.check() == sat:
    model = solver.model()
    print(f"Собаки: {model[x].as_long()}, Коты: {model[y].as_long()}, Мыши: {model[z].as_long()}")
else:
    print("Решений нет")


Собаки: 3, Коты: 41, Мыши: 56


Один из классических типов головоломок -- это нахождение цифр, соответствующих буквам словесного выражения. Например, имеется выражение send + more = money, и надо вместо букв подставить подходящие цифры (одинаковые буквы -- это одинаковые цифры), чтобы оно получилось истинным.

In [ ]:
from z3 import *

# Объявляем переменные
S, E, N, D, M, O, R, Y = Ints('S E N D M O R Y')

# Все переменные — цифры от 0 до 9
digits = [S, E, N, D, M, O, R, Y]

solver = Solver()
solver.add([And(d >= 0, d <= 9) for d in digits])
solver.add(Distinct(digits))  # все буквы — разные цифры

# Условие: S и M ≠ 0
solver.add(S != 0, M != 0)

# Основное уравнение:
send = 1000*S + 100*E + 10*N + D
more = 1000*M + 100*O + 10*R + E
money = 10000*M + 1000*O + 100*N + 10*E + Y

solver.add(send + more == money)

# Проверим решение
if solver.check() == sat:
    model = solver.model()
    result = {str(d): model[d].as_long() for d in digits}
    print("Решение:")
    for k in sorted(result):
        print(f"{k} = {result[k]}")
else:
    print("Решения нет")


## 8.3. Восемь ферзей
Ещё одна классическая задачка, которую очень любят решать на курсах по функциональному и декларативному программированию. На шахматной доске надо расставить восемь ферзей так, чтобы никто друг друга не атаковал.

In [ ]:
from z3 import *

# Один ферзь в каждой строке: переменные — столбцы (0–7) для каждой строки
queens = [Int(f'q_{i}') for i in range(8)]

solver = Solver()

# Каждая переменная — целое от 0 до 7 (столбец)
solver.add([And(q >= 0, q < 8) for q in queens])

# Все ферзи должны стоять в разных столбцах
solver.add(Distinct(queens))

# Ни один ферзь не должен бить другого по диагонали
for i in range(8):
    for j in range(i + 1, 8):
        solver.add(queens[i] != queens[j])                     # уже учтено в Distinct
        solver.add(Abs(queens[i] - queens[j]) != j - i)       # диагонали

# Решение
if solver.check() == sat:
    model = solver.model()
    result = [model.evaluate(q).as_long() for q in queens]
    print("Решение (индексы столбцов для строк 0–7):", result)
else:
    print("Решения нет")


## 7.2. Вторая задачка по физике

Мотоциклист стоит на светофоре. Как только загорается зелёный, он стартует с места с ускорением 6 м/с^2 на протяжении 4,1 секунды. Определите расстояние, которое мотоцикл преодолеет за это время.


In [ ]:
from z3 import *

t = Real('t')
s = Real('s')
a = 6  # ускорение
eq = s == 0.5 * a * t * t

solver = Solver()
solver.add(eq, t == 4.1)

if solver.check() == sat:
    m = solver.model()
    print(m[s].as_decimal(5))  # 5 знаков после запятой
